###### 2020-11-22 일요일
###### 2020-11-23 월요일

In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## K-Fold 적용하지 않음

# 01_텍스트데이터토큰화 & 토큰사전저장
 - 음절단위 토큰화 진행 과정에서 많은 시간이 소요된다
 - 그래서 의미없는 시간 손실을 줄이기 위해 음절단위 토큰화를 진행하고 음절단위 토큰사전을 json파일로 저장하고자 한다.

In [61]:
import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import re
import json
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

## 1. 데이터 불러오기

In [63]:
comment_df = pd.read_csv('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/comment_data.csv', encoding='UTF-8-SIG')
add_comment_df = pd.read_excel('/content/drive/MyDrive/[final_project]_악플원정대/raw_data/add_comment.csv')

comment_df = comment_df[ ['댓글', '악플여부'] ]

In [64]:
# 이 셀을 실행하면 추가한 데이터를 확인할 수 있음
# 데이터가 추가 되었는지 확인하고 진행하길 바람
add_comment_df.tail(20)

,댓글,악플여부
20377,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20378,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20379,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20380,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20381,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20382,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20383,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20384,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20385,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2
20386,남자들 정은 마치 들의 냇물 같아서 갈라지는 경우가 많지만 여자 마음은 외줄기 폭포...,2


In [65]:
# 하나의 데이터프레임으로 합치기
all_df = pd.concat([comment_df, add_comment_df])


In [66]:

# 라벨이 NaN인거 제거
index = all_df['악플여부'].isna()
raw_df = all_df[~index]

# 인덱스 리셋
raw_df = raw_df.reset_index()
raw_df.drop(['index'], inplace=True, axis=1)

In [67]:
# 원본유지 복사본 사용
copy_df = raw_df.copy()

In [68]:
# 고정 시드값 지정
seed = 123

# 댓글 길이 지정
comment_len = 400

#### 악플여부 라벨 1인 경우와 라벨 2인 경우를 하나로 합친다

In [69]:
index = copy_df['악플여부'] == 2
copy_df.loc[index, '악플여부'] = 1

In [70]:
copy_df['악플여부'] = copy_df['악플여부'].apply(int)
copy_df['댓글'] = copy_df['댓글'].apply(str)

In [71]:
copy_df['악플여부'].value_counts()

0    69552
1    40709
Name: 악플여부, dtype: int64

## 2. 토큰화 & 저장
 - Train set, Test set을 나눈다
 - Train set을 기준으로 음절 토큰 사전을 만든다
 - Train set / Test set / 음절토큰사전을 저장한다
    - 이렇게 저장하는 이유는? : 매번 음절토큰사전을 만드는데 시간이 많이 소요되기 때문에 한번 만들어놓고 재사용하여 시간을 절약!


In [72]:
def tokenization_save(df):

    feature = df['댓글']
    label = df['악플여부']

    X_train, X_test, y_train, y_test = train_test_split(feature, label, 
                                                    test_size=0.2,
                                                    random_state=seed)
    
    X_train_token_list = list(''.join(X_train))
    
    tokenizer = Tokenizer(oov_token = 'OOV')
    tokenizer.fit_on_texts(X_train_token_list)
    
    word_index_vocab = tokenizer.word_index

    return X_train, X_test, y_train, y_test, word_index_vocab


In [73]:
def train_test_save(X_train, X_test, y_train, y_test) :
    save_df_list = [X_train, X_test, y_train, y_test]
    file_names = ['X_train', 'X_test', 'y_train', 'y_test']
    for idx, set in enumerate(save_df_list):
        set.to_csv('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/' + file_names[idx] + '.csv', encoding='UTF-8-SIG')


def save_word_index(word_index_vocab) :
    json_file = json.dumps(word_index_vocab, ensure_ascii=False)
    f = open('/content/drive/MyDrive/[final_project]_악플원정대/01_K-Fold_X/data/word_index_vocab.json', 'w')
    f.write(json_file)
    f.close()

In [74]:
# 토큰화
start = time.time()

X_train, X_test, y_train, y_test, word_index_vocab = tokenization_save(copy_df)

print("걸린시간 :", time.time() - start)

걸린시간 : 17.544846773147583


In [75]:
# Train / Test 저장
train_test_save(X_train, X_test, y_train, y_test)

# 음절단위토큰 저장
save_word_index(word_index_vocab)